Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

In [ ]:
# Uncomment these if explanation packages are not already installed in your environment
#!pip install --upgrade azureml-sdk[explain]

Explain a model with the AML explain-model package

1. Train a SVM model using Scikit-learn
2. Run 'explain_model' in local mode, which doesn't contact any Azure services
3. Run 'explain_model' with AML Run History, which leverages Run History Service to store and manage the explanation data

# Disclaimer: this notebook is a preview of model explainability, and the APIs shown below are subject to breaking changes

## Train a SVM model, which we will try to explain

In [ ]:
# Import Iris dataset
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
# Split data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=0)

In [ ]:
# Import scikit learn, fit a SVM model
def create_scikit_learn_model(X, y):
    from sklearn import svm
    clf = svm.SVC(gamma=0.001, C=100., probability=True)
    model = clf.fit(X, y)
    return model
model = create_scikit_learn_model(x_train, y_train)

## Run model explainer locally

In [ ]:
from azureml.explain.model.tabular_explainer import TabularExplainer

In [ ]:
import time
start = time.time()

explainer = TabularExplainer(model, x_train, features=iris.feature_names)
global_explanation = explainer.explain_global(x_test)

# importance values for each class, test example, and feature (local importance)
local_imp_values = global_explanation.local_importance_values
# base prediction with feature importances ignored
expected_values = global_explanation.expected_values
# global feature importance information
global_imp_values = global_explanation.global_importance_values
ranked_global_imp_names = global_explanation.get_ranked_global_names()
# global per-class feature importance information
per_class_imp_values = global_explanation.per_class_values
ranked_per_class_imp_names = global_explanation.get_ranked_per_class_names()

end = time.time()
print(end - start)

## Run model explainer with AML Run History

In [ ]:
import azureml.core
from azureml.core import Workspace, Experiment, Run
from azureml.explain.model.tabular_explainer import TabularExplainer
from azureml.contrib.explain.model.explanation.explanation_client import ExplanationClient
# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

In [ ]:
experiment_name = 'explain_model'
experiment = Experiment(ws, experiment_name)
run = experiment.start_logging()
client = ExplanationClient.from_run(run)

In [ ]:
import time
start = time.time()
explainer = TabularExplainer(model, x_train, features=iris.feature_names, classes=iris.target_names)
explanation = explainer.explain_global(x_test)
client.upload_model_explanation(explanation)
end = time.time()
print(end - start)

In [ ]:
explanation_from_run = client.download_model_explanation()

In [ ]:
# global feature importance information
global_imp_values = explanation_from_run.global_importance_values
global_imp_names = explanation_from_run.get_ranked_global_names()
# global per-class feature importance information
per_class_imp_values = explanation_from_run.per_class_values
per_class_imp_names = explanation_from_run.get_ranked_per_class_names()

## This visualization is unsupported, and is not guaranteed to work in the future

In [ ]:
# Get the shap values and explore locally
import shap
import numpy as np
shap.initjs()
display(shap.force_plot(explanation_from_run.expected_values[1], np.asarray(explanation_from_run.local_importance_values[1]), x_test))

In [ ]:
run.complete()